# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import numpy as np
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
import pickle
import warnings

warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)

X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detect_urls = re.findall(url_regex, text)
    for url in detect_urls:
        text = text.replace(url, "urlplaceholder")
    # take punctuation 
    tokenizer = RegexpTokenizer(r'\w+')
    token = tokenizer.tokenize(text)
    # lemmatize
    lemmatizer = WordNetLemmatizer()
    clean_token = []
    for t in token:
        tokens = lemmatizer.lemmatize(t).lower().strip()
        clean_token.append(tokens)      
    return clean_token

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [20]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeClassifier()), 
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline2.fit(X_train,y_train)
y_pred_test = pipeline2.predict(X_test)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:

# print the f1 score, precision and recall for each output category of the dataset.
#print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
#print("*******************************************************************************")
#print(classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

# Printing the f1 score, precision and recall for each output category of the dataset by iterating through the columns
def print_results(y_test, y_pred):
    
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    counter = 0
    
    for i in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[i], y_pred[:,counter], average='weighted')
        results.set_value(counter+1, 'Category', i)
        results.set_value(counter+1, 'f_score', f_score)
        results.set_value(counter+1, 'precision', precision)
        results.set_value(counter+1, 'recall', recall)
        counter += 1
        
    print('f_score:', results['f_score'].mean())
    print('precision:', results['precision'].mean())
    print('recall:', results['recall'].mean())
    
    return results



In [22]:
results = print_results(y_test, y_pred)
results

f_score: 0.892217639676
precision: 0.877853631398
recall: 0.910194791985


,Category,f_score,precision,recall
1,related,0.655048,0.633898,0.686451
2,request,0.744832,0.719036,0.780439
3,offer,0.994282,0.992386,0.996186
4,aid_related,0.506671,0.502885,0.521666
5,medical_help,0.883289,0.858927,0.915014
6,medical_products,0.933227,0.917271,0.952548
7,search_and_rescue,0.958904,0.947532,0.970552
8,security,0.973376,0.965202,0.981691
9,military,0.949445,0.936772,0.962466
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
cv.best_params_

{'clf__estimator__max_depth': None, 'clf__estimator__min_samples_leaf': 2}

In [31]:
cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__vect', 'estimator__tfidf', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__preprocessor', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__vocabulary', 'estimator__tfidf__norm', 'estimator__tfidf__smooth_idf', 'estimator__tfidf__sublinear_tf', 'estimator__tfidf__use_idf', 'estimator__clf__estimator__bootstrap', 'estimator__clf__estimator__class_weight', 'estimator__clf__estimator__criterion', 'estimator__clf__estimator__max_depth', 'estimator__clf__estimator__max_features', 'estimator__clf__estimator__max

In [25]:
parameters = {'clf__estimator__max_depth': [10, 40, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
cv.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = cv.predict(X_test)


In [27]:
improved_model = print_results(y_test, y_pred)
improved_model

f_score: 0.926871883752
precision: 0.934331089062
recall: 0.942532973926


,Category,f_score,precision,recall
1,related,0.75698,0.785502,0.798139
2,request,0.851623,0.875398,0.876564
3,offer,0.994282,0.992386,0.996186
4,aid_related,0.740325,0.74134,0.743363
5,medical_help,0.894806,0.905469,0.924779
6,medical_products,0.939969,0.942513,0.95682
7,search_and_rescue,0.960733,0.974301,0.973604
8,security,0.973758,0.965215,0.982453
9,military,0.956617,0.964826,0.969484
10,child_alone,1,1,1


In [28]:
# Best mean test score
np.max(cv.cv_results_['mean_test_score'])

0.20114942528735633

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [19]:
model_results = print_results(y_test, y_pred)
model_results

f_score: 0.928643900619
precision: 0.931712083137
recall: 0.942469399519


,Category,f_score,precision,recall
1,related,0.779692,0.78008,0.799664
2,request,0.853939,0.873193,0.875343
3,offer,0.994282,0.992386,0.996186
4,aid_related,0.725287,0.735609,0.73375
5,medical_help,0.889322,0.890568,0.919591
6,medical_products,0.928402,0.942524,0.949496
7,search_and_rescue,0.96186,0.959436,0.972688
8,security,0.97631,0.974673,0.983674
9,military,0.960117,0.959877,0.970247
10,child_alone,1,1,1


### 9. Export your model as a pickle file

In [71]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.